# Fantasy Football Advanced Visualizations

This notebook implements 4 core advanced visualizations for fantasy football draft decision-making:

1. **VBD Bar Chart with Delta Overlays** - Horizontal bars showing VBD with red overlay showing value loss if you wait
2. **Availability Sparkline Grid** - Grid of mini probability charts for quick scanning (4x5 grid)
3. **Decision Score Gauges** - Plotly gauge charts showing 0-100 decision scores for top players
4. **Position Scarcity Thermometers** - Vertical bars for each position showing remaining elite players

Each visualization is designed to provide immediate actionable insights for draft decisions.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

# Set plotting styles
plt.style.use('default')
sns.set_palette("husl")
print("✅ All libraries imported successfully")

✅ All libraries imported successfully


## Data Loading and Processing

Load data from existing CSV files and integrate with the 80% ESPN + 20% ADP probability system.

In [2]:
def load_and_merge_ranking_data():
    """Load ESPN rankings and external ADP data, merge them for probability calculations"""
    
    # Load ESPN data (80% weight)
    espn_df = pd.read_csv('data/espn_projections_20250814.csv')
    
    # Load FantasyPros ADP data (20% weight)  
    adp_df = pd.read_csv('data/fantasypros_adp_20250815.csv')
    
    print(f"ESPN data: {len(espn_df)} players")
    print(f"ADP data: {len(adp_df)} players")
    
    # Standardize player names for matching
    def clean_name(name):
        return str(name).strip().lower().replace("'", "").replace(".", "")
    
    espn_df['clean_name'] = espn_df['player_name'].apply(clean_name)
    adp_df['clean_name'] = adp_df['PLAYER'].apply(clean_name)
    
    # Merge ESPN with ADP data
    merged_df = espn_df.merge(
        adp_df[['clean_name', 'RANK', 'ADP']], 
        on='clean_name', 
        how='left'
    )
    
    # Fill missing ADP ranks with ESPN rank + some penalty
    merged_df['adp_rank'] = merged_df['RANK'].fillna(merged_df['overall_rank'] + 50)
    merged_df['espn_rank'] = merged_df['overall_rank']
    
    print(f"Merged dataset: {len(merged_df)} players")
    print(f"Players with ADP data: {len(merged_df[merged_df['RANK'].notna()])} players")
    
    return merged_df[['overall_rank', 'position', 'position_rank', 'player_name', 'team', 'salary_value', 'bye_week', 'espn_rank', 'adp_rank']]

def load_vbd_data():
    """Load custom VBD rankings"""
    return pd.read_csv('draft_cheat_sheet.csv')

def compute_softmax_scores(rank_series, tau=5.0):
    """Convert ranks to softmax scores with temperature tau (lower rank = higher score)"""
    scores = np.exp(-rank_series / tau)
    return scores

def compute_pick_probabilities(available_df, espn_weight=0.8, adp_weight=0.2, tau_espn=5.0, tau_adp=5.0):
    """
    Compute per-player probability of being picked next using weighted softmax over ESPN and ADP ranks
    """
    if len(available_df) == 0:
        return pd.Series(dtype=float)
    
    # Softmax scores for ESPN and ADP ranks
    espn_scores = compute_softmax_scores(available_df['espn_rank'], tau_espn)
    adp_scores = compute_softmax_scores(available_df['adp_rank'], tau_adp)
    
    # Weighted combination (80% ESPN, 20% ADP)
    combined_scores = espn_weight * espn_scores + adp_weight * adp_scores
    
    # Normalize to sum to 1 across available players
    probs = combined_scores / combined_scores.sum()
    return probs

def probability_gone_before_next_pick(available_df, player_name, picks_until_next_turn):
    """
    Compute probability a player is gone before your next turn using discrete survival calculation
    """
    if picks_until_next_turn <= 0:
        return 0.0
    
    survival_prob = 1.0
    current_available = available_df.copy()
    
    # Simulate each pick until our next turn
    for pick_step in range(picks_until_next_turn):
        if len(current_available) == 0:
            break
            
        # Get probabilities for current available players
        pick_probs = compute_pick_probabilities(current_available)
        
        # Find probability our target player gets picked this round
        player_mask = current_available['player_name'] == player_name
        if not player_mask.any():
            # Player already gone
            break
            
        p_pick_now = pick_probs[player_mask].iloc[0] if player_mask.any() else 0.0
        
        # Update survival probability
        survival_prob *= (1 - p_pick_now)
        
        # For simulation: remove the most likely pick (simplified approach)
        most_likely_idx = pick_probs.idxmax()
        current_available = current_available.drop(most_likely_idx)
    
    prob_gone = 1 - survival_prob
    return min(1.0, max(0.0, prob_gone))

# Load and prepare data
print("Loading data...")
ranking_df = load_and_merge_ranking_data()
vbd_data = load_vbd_data()

print("\n✅ Data loaded successfully")

Loading data...
ESPN data: 300 players
ADP data: 347 players
Merged dataset: 300 players
Players with ADP data: 252 players

✅ Data loaded successfully


In [3]:
def calculate_enhanced_metrics(ranking_df, vbd_df, my_picks=None, drafted_players=None, current_pick=1):
    """Calculate enhanced metrics using new 80% ESPN + 20% ADP probability system"""
    if my_picks is None:
        my_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # Default pick positions
    if drafted_players is None:
        drafted_players = set()
    
    # Get next pick logic
    next_pick = None
    picks_to_next = None
    pick_after_next = None
    
    for pick in my_picks:
        if pick >= current_pick:
            next_pick = pick
            picks_to_next = pick - current_pick
            remaining_picks = [p for p in my_picks if p > next_pick]
            pick_after_next = remaining_picks[0] if remaining_picks else None
            break
    
    # Filter out drafted players from ranking data
    available_ranking_df = ranking_df[~ranking_df['player_name'].isin(drafted_players)].copy()
    
    # Merge with VBD data for VBD scores
    vbd_lookup = dict(zip(vbd_df['Player'], vbd_df['Custom_VBD']))
    available_ranking_df['vbd_score'] = available_ranking_df['player_name'].map(vbd_lookup)
    available_ranking_df['vbd_score'] = available_ranking_df['vbd_score'].fillna(0)
    
    enhanced_data = []
    
    # Position scarcity data
    position_scarcity = {}
    for pos in ['QB', 'RB', 'WR', 'TE']:
        pos_players = available_ranking_df[available_ranking_df['position'] == pos]
        elite_count = len(pos_players[pos_players['vbd_score'] >= 40])
        position_scarcity[pos] = elite_count
    
    for _, player in available_ranking_df.iterrows():
        player_name = player['player_name']
        espn_rank = player['espn_rank']
        adp_rank = player['adp_rank']
        vbd_score = player['vbd_score']
        
        # Calculate probabilities using discrete survival method
        prob_gone_by_next = probability_gone_before_next_pick(
            available_ranking_df, player_name, picks_to_next
        ) if picks_to_next > 0 else 0.0
        
        prob_available_at_next = 1 - prob_gone_by_next
        
        # Calculate probability gone by pick after next
        picks_to_after = (pick_after_next - current_pick) if pick_after_next else 0
        prob_gone_by_after = probability_gone_before_next_pick(
            available_ranking_df, player_name, picks_to_after
        ) if picks_to_after > 0 else 0.0
        
        prob_available_at_after = 1 - prob_gone_by_after
        
        # Decision score using VBD × availability probability
        decision_score = vbd_score * prob_available_at_next
        
        # Value loss if wait (for delta overlay)
        value_loss_if_wait = vbd_score * prob_gone_by_next
        
        # Opportunity cost (difference from next best at position)
        pos_players = available_ranking_df[available_ranking_df['position'] == player['position']]
        better_players = pos_players[pos_players['vbd_score'] > vbd_score].sort_values('vbd_score', ascending=False)
        if len(better_players) > 0:
            next_best_vbd = better_players.iloc[0]['vbd_score']
            opportunity_cost = max(0, next_best_vbd - vbd_score)
        else:
            opportunity_cost = 0
        
        # Decision logic
        if prob_available_at_next > 0.8:
            decision_notes = f"SAFE - {prob_available_at_next:.0%} chance available at pick {next_pick}"
        elif prob_available_at_after and prob_available_at_after > 0.7:
            decision_notes = f"WAIT - Target at pick {pick_after_next}"
        elif prob_available_at_next > 0.3:
            decision_notes = f"DRAFT NOW - Only {prob_available_at_next:.0%} chance available later"
        else:
            decision_notes = f"REACH - Must draft at pick {next_pick} to secure"
        
        # Add VBD tier
        if vbd_score >= 100:
            decision_notes = "ELITE - " + decision_notes
        elif vbd_score >= 70:
            decision_notes = "STRONG - " + decision_notes
        elif vbd_score >= 40:
            decision_notes = "SOLID - " + decision_notes
        else:
            decision_notes = "DEPTH - " + decision_notes
        
        enhanced_data.append({
            'player_name': player_name,
            'position': player['position'],
            'position_rank': player['position_rank'],
            'espn_rank': int(espn_rank),
            'adp_rank': int(adp_rank),
            'vbd_score': vbd_score,
            'team': player['team'],
            'bye_week': player['bye_week'],
            'current_pick': current_pick,
            'next_pick': next_pick,
            'picks_to_next': picks_to_next,
            'pick_after_next': pick_after_next,
            'prob_available_at_next': prob_available_at_next,
            'prob_available_at_after': prob_available_at_after,
            'decision_score': decision_score,
            'value_loss_if_wait': value_loss_if_wait,
            'opportunity_cost': opportunity_cost,
            'decision_notes': decision_notes,
            'position_scarcity': position_scarcity.get(player['position'], 0)
        })
    
    enhanced_df = pd.DataFrame(enhanced_data)
    
    print(f"Enhanced data calculated for {len(enhanced_df)} available players")
    print(f"Current pick: {current_pick} | Next pick: {next_pick} (in {picks_to_next} picks)")
    
    return enhanced_df, position_scarcity

# Calculate enhanced metrics
my_draft_picks = [8, 17, 32, 41, 56, 65, 80, 89]
current_draft_pick = 1
drafted_players_list = set()  # Empty for now

enhanced_df, position_scarcity = calculate_enhanced_metrics(
    ranking_df, vbd_data, 
    my_picks=my_draft_picks, 
    current_pick=current_draft_pick,
    drafted_players=drafted_players_list
)

print(f"\n✅ Enhanced metrics calculated")
print(f"Position scarcity: {position_scarcity}")

Enhanced data calculated for 300 available players
Current pick: 1 | Next pick: 8 (in 7 picks)

✅ Enhanced metrics calculated
Position scarcity: {'QB': 5, 'RB': 16, 'WR': 9, 'TE': 3}


## Visualization 1: VBD Bar Chart with Delta Overlays

Horizontal bars showing VBD scores with red overlay indicating value loss if you wait until next pick.

In [4]:
def create_vbd_bar_chart_with_deltas(enhanced_df, top_n=15):
    """
    Create horizontal bar chart showing VBD scores with red overlay showing value loss if you wait
    """
    # Get top players by VBD score
    df_viz = enhanced_df.nlargest(top_n, 'vbd_score').copy()
    
    # Create figure with plotly
    fig = go.Figure()
    
    # Add VBD score bars (main bars)
    fig.add_trace(go.Bar(
        y=df_viz['player_name'],
        x=df_viz['vbd_score'],
        name='VBD Score',
        orientation='h',
        marker_color='rgba(26, 118, 255, 0.8)',
        text=[f"{score:.1f}" for score in df_viz['vbd_score']],
        textposition='inside',
        textfont=dict(color='white', size=10)
    ))
    
    # Add value loss overlay (red bars showing potential loss)
    fig.add_trace(go.Bar(
        y=df_viz['player_name'],
        x=df_viz['value_loss_if_wait'],
        name='Value Loss If Wait',
        orientation='h',
        marker_color='rgba(255, 65, 54, 0.9)',
        text=[f"-{loss:.1f}" for loss in df_viz['value_loss_if_wait']],
        textposition='inside',
        textfont=dict(color='white', size=9, family='Arial Black')
    ))
    
    # Update layout
    fig.update_layout(
        title={
            'text': f"VBD Scores with Value Loss Risk (Top {top_n} Players)",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16, 'family': 'Arial Black'}
        },
        xaxis_title="VBD Score / Value Loss",
        yaxis_title="Player",
        height=max(400, top_n * 25),
        barmode='overlay',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template='plotly_white',
        margin=dict(l=150, r=50, t=80, b=50)
    )
    
    # Add annotations explaining the overlay
    fig.add_annotation(
        text="Red overlay shows expected value loss if player is drafted by others before your next pick",
        xref="paper", yref="paper",
        x=0.5, y=-0.1,
        showarrow=False,
        font=dict(size=12, color="gray"),
        xanchor='center'
    )
    
    return fig

# Create and display the VBD bar chart with deltas
vbd_delta_chart = create_vbd_bar_chart_with_deltas(enhanced_df, top_n=15)
vbd_delta_chart.show()

print("📊 VBD Bar Chart with Delta Overlays created")
print("Blue bars = Total VBD value | Red overlay = Value at risk if you wait")

📊 VBD Bar Chart with Delta Overlays created
Blue bars = Total VBD value | Red overlay = Value at risk if you wait


## Visualization 2: Availability Sparkline Grid

Grid of mini probability charts (4x5 grid) showing availability probability across draft picks for quick scanning.

In [5]:
def create_availability_sparkline_grid(enhanced_df, grid_size=(4, 5)):
    """
    Create a grid of sparklines showing availability probability for top players
    """
    rows, cols = grid_size
    total_players = rows * cols
    
    # Get top players by VBD score
    df_viz = enhanced_df.nlargest(total_players, 'vbd_score').copy()
    
    # Create subplots
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=[f"{row['player_name']} ({row['position']})" for _, row in df_viz.iterrows()],
        vertical_spacing=0.08,
        horizontal_spacing=0.05
    )
    
    # Generate sparkline data for each player
    for idx, (_, player) in enumerate(df_viz.iterrows()):
        row = (idx // cols) + 1
        col = (idx % cols) + 1
        
        # Generate availability probability across picks
        pick_range = list(range(1, 25))  # First 24 picks
        availability_probs = []
        
        for pick in pick_range:
            prob = 1 - stats.norm.cdf(pick, loc=player['espn_rank'], scale=3)
            availability_probs.append(max(0, min(1, prob)))
        
        # Add sparkline
        fig.add_trace(
            go.Scatter(
                x=pick_range,
                y=availability_probs,
                mode='lines+markers',
                line=dict(color='#1f77b4', width=2),
                marker=dict(size=3),
                name=player['player_name'],
                showlegend=False
            ),
            row=row, col=col
        )
        
        # Highlight your next pick positions
        next_pick = player['next_pick']
        if next_pick and next_pick <= 24:
            next_pick_prob = 1 - stats.norm.cdf(next_pick, loc=player['espn_rank'], scale=3)
            fig.add_trace(
                go.Scatter(
                    x=[next_pick],
                    y=[next_pick_prob],
                    mode='markers',
                    marker=dict(color='red', size=8, symbol='diamond'),
                    name='Your Next Pick',
                    showlegend=(idx == 0)  # Only show legend for first trace
                ),
                row=row, col=col
            )
        
        # Color code background based on availability at next pick
        if next_pick:
            prob_at_next = player['prob_available_at_next']
            if prob_at_next > 0.8:
                bg_color = 'rgba(46, 204, 113, 0.1)'  # Green
            elif prob_at_next > 0.5:
                bg_color = 'rgba(241, 196, 15, 0.1)'  # Yellow
            else:
                bg_color = 'rgba(231, 76, 60, 0.1)'   # Red
            
            fig.add_shape(
                type="rect",
                xref=f"x{idx+1}", yref=f"y{idx+1}",
                x0=1, y0=0, x1=24, y1=1,
                fillcolor=bg_color,
                layer="below",
                line_width=0,
                row=row, col=col
            )
    
    # Update layout
    fig.update_layout(
        title={
            'text': "Availability Sparkline Grid - Top 20 Players",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16, 'family': 'Arial Black'}
        },
        height=800,
        template='plotly_white',
        showlegend=True
    )
    
    # Update axes for all subplots
    for i in range(1, total_players + 1):
        fig.update_xaxes(
            title="Pick #" if i > (rows-1)*cols else "",
            range=[1, 24],
            showgrid=True,
            gridwidth=0.5,
            row=(i-1)//cols + 1, 
            col=(i-1)%cols + 1
        )
        fig.update_yaxes(
            title="Avail %" if (i-1) % cols == 0 else "",
            range=[0, 1],
            tickformat=".0%",
            showgrid=True,
            gridwidth=0.5,
            row=(i-1)//cols + 1, 
            col=(i-1)%cols + 1
        )
    
    return fig

# Create and display sparkline grid
sparkline_grid = create_availability_sparkline_grid(enhanced_df, grid_size=(4, 5))
sparkline_grid.show()

print("📊 Availability Sparkline Grid created")
print("Red diamonds = Your next pick | Background color = Risk level (Green=Safe, Yellow=Risky, Red=Danger)")

📊 Availability Sparkline Grid created
Red diamonds = Your next pick | Background color = Risk level (Green=Safe, Yellow=Risky, Red=Danger)


## Visualization 3: Decision Score Gauges

Plotly gauge charts showing 0-100 decision scores for top players, combining VBD value and availability probability.

In [6]:
def create_decision_score_gauges(enhanced_df, top_n=8):
    """
    Create gauge charts showing decision scores (0-100) for top players
    """
    # Get top players by decision score
    df_viz = enhanced_df.nlargest(top_n, 'decision_score').copy()
    
    # Normalize decision scores to 0-100 scale
    max_decision_score = df_viz['decision_score'].max()
    df_viz['normalized_score'] = (df_viz['decision_score'] / max_decision_score * 100).round(1)
    
    # Create subplots for gauges
    rows = 2
    cols = 4
    fig = make_subplots(
        rows=rows, cols=cols,
        specs=[[{'type': 'indicator'}] * cols for _ in range(rows)],
        subplot_titles=[f"{row['player_name']} ({row['position']})" for _, row in df_viz.iterrows()],
        vertical_spacing=0.25,
        horizontal_spacing=0.1
    )
    
    # Add gauge for each player
    for idx, (_, player) in enumerate(df_viz.iterrows()):
        row = (idx // cols) + 1
        col = (idx % cols) + 1
        
        normalized_score = player['normalized_score']
        
        # Determine color based on score ranges
        if normalized_score >= 80:
            gauge_color = "#2ecc71"  # Green
            threshold_color = "#27ae60"
        elif normalized_score >= 60:
            gauge_color = "#f39c12"  # Orange
            threshold_color = "#e67e22"
        elif normalized_score >= 40:
            gauge_color = "#3498db"  # Blue
            threshold_color = "#2980b9"
        else:
            gauge_color = "#95a5a6"  # Gray
            threshold_color = "#7f8c8d"
        
        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=normalized_score,
                domain={'x': [0, 1], 'y': [0, 1]},
                title={'text': f"VBD: {player['vbd_score']:.1f}<br>Avail: {player['prob_available_at_next']:.0%}"},
                delta={'reference': 50, 'valueformat': '.1f'},
                gauge={
                    'axis': {'range': [None, 100]},
                    'bar': {'color': gauge_color},
                    'steps': [
                        {'range': [0, 40], 'color': "#ecf0f1"},
                        {'range': [40, 60], 'color': "#bdc3c7"},
                        {'range': [60, 80], 'color': "#95a5a6"}
                    ],
                    'threshold': {
                        'line': {'color': threshold_color, 'width': 4},
                        'thickness': 0.75,
                        'value': normalized_score
                    }
                },
                number={'font': {'size': 16, 'family': 'Arial Black'}}
            ),
            row=row, col=col
        )
    
    # Update layout
    fig.update_layout(
        title={
            'text': f"Decision Score Gauges - Top {top_n} Players by Decision Score",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16, 'family': 'Arial Black'}
        },
        height=600,
        template='plotly_white',
        font={'family': 'Arial', 'size': 12}
    )
    
    # Add explanation annotation
    fig.add_annotation(
        text="Decision Score = VBD Value × Availability Probability (normalized to 0-100)<br>Green=Draft Now, Orange=Consider, Blue=Monitor, Gray=Low Priority",
        xref="paper", yref="paper",
        x=0.5, y=-0.1,
        showarrow=False,
        font=dict(size=12, color="gray"),
        xanchor='center'
    )
    
    return fig

# Create and display decision score gauges
decision_gauges = create_decision_score_gauges(enhanced_df, top_n=8)
decision_gauges.show()

print("📊 Decision Score Gauges created")
print("Decision Score = VBD Value × Availability Probability")
print("Green=Draft Now (80+), Orange=Consider (60-79), Blue=Monitor (40-59), Gray=Low Priority (<40)")

📊 Decision Score Gauges created
Decision Score = VBD Value × Availability Probability
Green=Draft Now (80+), Orange=Consider (60-79), Blue=Monitor (40-59), Gray=Low Priority (<40)


## Visualization 4: Position Scarcity Thermometers

Vertical thermometer-style bars for each position showing remaining elite players and scarcity levels.

In [7]:
def create_position_scarcity_thermometers(enhanced_df, position_scarcity):
    """
    Create thermometer-style visualization showing position scarcity
    """
    positions = ['QB', 'RB', 'WR', 'TE']
    
    # Calculate detailed scarcity metrics for each position
    scarcity_data = []
    
    for pos in positions:
        pos_players = enhanced_df[enhanced_df['position'] == pos].copy()
        
        # Count players by VBD tiers
        elite_count = len(pos_players[pos_players['vbd_score'] >= 80])  # Elite tier
        strong_count = len(pos_players[pos_players['vbd_score'] >= 60])  # Strong tier
        solid_count = len(pos_players[pos_players['vbd_score'] >= 40])  # Solid tier
        total_count = len(pos_players)
        
        # Calculate scarcity temperature (higher = more scarce)
        if elite_count == 0:
            scarcity_temp = 100
        else:
            scarcity_temp = max(0, 100 - (elite_count * 20))  # Each elite player reduces scarcity by 20
        
        scarcity_data.append({
            'position': pos,
            'elite_count': elite_count,
            'strong_count': strong_count,
            'solid_count': solid_count,
            'total_count': total_count,
            'scarcity_temp': scarcity_temp
        })
    
    # Create thermometer visualization using plotly
    fig = go.Figure()
    
    # Define position colors
    position_colors = {
        'QB': '#9b59b6',  # Purple
        'RB': '#e74c3c',  # Red
        'WR': '#3498db',  # Blue
        'TE': '#f39c12'   # Orange
    }
    
    # Create thermometer bars
    for i, data in enumerate(scarcity_data):
        pos = data['position']
        temp = data['scarcity_temp']
        
        # Thermometer base (gray background)
        fig.add_trace(go.Bar(
            x=[pos],
            y=[100],
            name=f'{pos} Base',
            marker_color='#ecf0f1',
            showlegend=False,
            width=0.4
        ))
        
        # Thermometer fill (colored by scarcity level)
        if temp >= 80:
            fill_color = '#e74c3c'  # Red - Very Scarce
        elif temp >= 60:
            fill_color = '#f39c12'  # Orange - Scarce
        elif temp >= 40:
            fill_color = '#f1c40f'  # Yellow - Moderate
        else:
            fill_color = '#27ae60'  # Green - Abundant
        
        fig.add_trace(go.Bar(
            x=[pos],
            y=[temp],
            name=f'{pos} Scarcity',
            marker_color=fill_color,
            showlegend=False,
            width=0.35,
            text=f"{temp:.0f}°<br>Elite: {data['elite_count']}<br>Strong: {data['strong_count']}<br>Total: {data['total_count']}",
            textposition='inside',
            textfont=dict(color='white', size=10, family='Arial Black')
        ))
    
    # Add scarcity level indicators
    scarcity_levels = [
        {'level': 'CRITICAL', 'range': [80, 100], 'color': '#e74c3c'},
        {'level': 'HIGH', 'range': [60, 80], 'color': '#f39c12'},
        {'level': 'MODERATE', 'range': [40, 60], 'color': '#f1c40f'},
        {'level': 'LOW', 'range': [0, 40], 'color': '#27ae60'}
    ]
    
    # Update layout
    fig.update_layout(
        title={
            'text': "Position Scarcity Thermometers - Elite Player Availability",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16, 'family': 'Arial Black'}
        },
        xaxis_title="Position",
        yaxis_title="Scarcity Temperature (°)",
        yaxis=dict(
            range=[0, 100],
            tickmode='linear',
            tick0=0,
            dtick=20
        ),
        height=500,
        template='plotly_white',
        barmode='overlay'
    )
    
    # Add temperature zone annotations
    zone_annotations = [
        {'y': 90, 'text': 'CRITICAL', 'color': '#e74c3c'},
        {'y': 70, 'text': 'HIGH', 'color': '#f39c12'},
        {'y': 50, 'text': 'MODERATE', 'color': '#f1c40f'},
        {'y': 20, 'text': 'LOW', 'color': '#27ae60'}
    ]
    
    for zone in zone_annotations:
        fig.add_annotation(
            x=-0.5,
            y=zone['y'],
            text=zone['text'],
            showarrow=False,
            font=dict(color=zone['color'], size=10, family='Arial Black'),
            xanchor='center'
        )
    
    # Add explanation
    fig.add_annotation(
        text="Higher temperature = More scarce position. Numbers show Elite/Strong/Total players remaining.",
        xref="paper", yref="paper",
        x=0.5, y=-0.15,
        showarrow=False,
        font=dict(size=12, color="gray"),
        xanchor='center'
    )
    
    return fig, scarcity_data

# Create and display position scarcity thermometers
scarcity_thermometers, scarcity_summary = create_position_scarcity_thermometers(enhanced_df, position_scarcity)
scarcity_thermometers.show()

print("📊 Position Scarcity Thermometers created")
print("\nScarcity Summary:")
for data in scarcity_summary:
    print(f"{data['position']}: {data['scarcity_temp']:.0f}° - Elite: {data['elite_count']}, Strong: {data['strong_count']}, Total: {data['total_count']}")

📊 Position Scarcity Thermometers created

Scarcity Summary:
QB: 60° - Elite: 2, Strong: 4, Total: 32
RB: 0° - Elite: 6, Strong: 12, Total: 90
WR: 60° - Elite: 2, Strong: 5, Total: 104
TE: 80° - Elite: 1, Strong: 3, Total: 34


## Interactive Dashboard Controls

Widget controls to update draft position and see how visualizations change in real-time.

In [8]:
def create_interactive_dashboard_controls():
    """
    Create interactive controls to update draft scenario and refresh visualizations
    """
    # Create widgets
    current_pick_slider = widgets.IntSlider(
        value=1,
        min=1,
        max=50,
        step=1,
        description='Current Pick:',
        style={'description_width': 'initial'}
    )
    
    drafted_players_text = widgets.Textarea(
        value='',
        placeholder='Enter drafted player names, one per line',
        description='Drafted Players:',
        rows=3,
        style={'description_width': 'initial'}
    )
    
    refresh_button = widgets.Button(
        description='🔄 Refresh Visualizations',
        button_style='primary',
        tooltip='Click to update all visualizations with current settings'
    )
    
    output = widgets.Output()
    
    def refresh_visualizations(b):
        """Refresh all visualizations with current settings"""
        with output:
            clear_output(wait=True)
            
            try:
                # Parse drafted players
                drafted_text = drafted_players_text.value.strip()
                drafted_set = set(line.strip() for line in drafted_text.split('\n') if line.strip())
                
                print(f"🔄 Refreshing visualizations...")
                print(f"Current pick: {current_pick_slider.value}")
                print(f"Drafted players: {len(drafted_set)} players")
                print("=" * 50)
                
                # Recalculate enhanced metrics
                enhanced_df_new, position_scarcity_new = calculate_enhanced_metrics(
                    ranking_df, vbd_data,
                    my_picks=my_draft_picks,
                    current_pick=current_pick_slider.value,
                    drafted_players=drafted_set
                )
                
                # Create updated visualizations
                print("\n📊 Creating updated VBD Bar Chart...")
                vbd_chart = create_vbd_bar_chart_with_deltas(enhanced_df_new, top_n=12)
                vbd_chart.show()
                
                print("\n📊 Creating updated Decision Score Gauges...")
                gauge_chart = create_decision_score_gauges(enhanced_df_new, top_n=8)
                gauge_chart.show()
                
                print("\n📊 Creating updated Position Scarcity Thermometers...")
                thermo_chart, _ = create_position_scarcity_thermometers(enhanced_df_new, position_scarcity_new)
                thermo_chart.show()
                
                print("\n✅ All visualizations updated successfully!")
                
            except Exception as e:
                print(f"❌ Error refreshing visualizations: {e}")
    
    # Connect button to refresh function
    refresh_button.on_click(refresh_visualizations)
    
    # Create layout
    controls = widgets.VBox([
        widgets.HTML("<h3>🎮 Interactive Dashboard Controls</h3>"),
        current_pick_slider,
        drafted_players_text,
        refresh_button
    ])
    
    return controls, output

# Create and display interactive controls
dashboard_controls, dashboard_output = create_interactive_dashboard_controls()

print("🎮 Interactive Dashboard Controls")
print("Update current pick and drafted players, then click refresh to update all visualizations")
print("=" * 80)

display(dashboard_controls)
display(dashboard_output)

🎮 Interactive Dashboard Controls
Update current pick and drafted players, then click refresh to update all visualizations


Output()

## Summary

This notebook provides 4 core advanced visualizations for fantasy football draft decision-making:

### ✅ Implemented Visualizations:

1. **VBD Bar Chart with Delta Overlays**
   - Horizontal bars showing VBD scores
   - Red overlay indicating value loss if you wait
   - Helps identify high-value players at risk

2. **Availability Sparkline Grid** 
   - 4x5 grid of mini probability charts
   - Shows availability trends across draft picks
   - Background colors indicate risk levels
   - Red diamonds mark your next pick

3. **Decision Score Gauges**
   - Gauge charts showing 0-100 decision scores
   - Combines VBD value × availability probability
   - Color coded by priority (Green=Draft Now, Orange=Consider, etc.)

4. **Position Scarcity Thermometers**
   - Vertical thermometer bars for QB/RB/WR/TE
   - Shows remaining elite players by position
   - Temperature indicates scarcity level

### 🎮 Interactive Features:
- Real-time updates based on current pick
- Drafted player tracking
- Refresh button to update all visualizations

### 🔧 Technical Implementation:
- Integrates with existing 80% ESPN + 20% ADP probability system
- Uses plotly for interactive charts
- Handles data from existing CSV files
- Includes error handling and user feedback

All visualizations are designed to provide immediate, actionable insights for fantasy football draft decisions.